inference

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from model import *
from data import *
from utils import *

pth_save_path = './example/ads/pth/'

mol2input = Mol2Input()
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
fitting_model = FittingNet(output_dim=1)
unimol_model = UniMolModel(output_dim=1, data_type='molecule', remove_hs=False)
fitting_model.load_state_dict(torch.load(pth_save_path +'atomic_fit_nh.pth'))
unimol_model.load_state_dict(torch.load(pth_save_path + 'atomic_model_nh.pth'))
fitting_model.to(device)
fitting_model.eval()
unimol_model.to(device)
unimol_model.eval()
print('Load model successfully!')

2024-11-28 06:01:29.274512: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 06:01:29.310911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 06:01:29.310940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 06:01:29.311998: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 06:01:29.318683: I tensorflow/core/platform/cpu_feature_guar

fused_multi_tensor is not installed corrected
fused_layer_norm is not installed corrected
fused_softmax is not installed corrected


2024-11-28 06:01:32 | model/unimol.py | 114 | INFO | Uni-Mol(QSAR) | Loading pretrained weights from /vepfs/fs_users/ycjin/ads_predict_tools/weights/mol_pre_all_h_220816.pt


Load model successfully!


单项预测

In [ ]:
from ase import Atoms
from ase.io import read,write
import py3Dmol
from ase.io import read
from rdkit import Chem
from rdkit.Chem import AllChem

# atom = read('temp.xyz')
input_file = './example/pub25/data/xyz/400.xyz' # input molecule file
atom = read(input_file)

# 去除H原子
sym = np.array(atom.get_chemical_symbols())
coord = [torch.tensor(atom.get_positions())[sym != 'H']]
atype = [np.array(atom.get_chemical_symbols())[sym != 'H']]
atom = atom[sym != 'H']

# 预测
input_dict = mol2input.coord2unimol_inputs(coord,atype)
for k in input_dict.keys(): input_dict[k] = input_dict[k].to(device)
atomic_reprs = unimol_model(return_repr=True,**input_dict)['atomic_reprs']
pred = []
for repr in atomic_reprs:
    atomic_p = fitting_model(repr)
    p = torch.sum(fitting_model(repr))

print(atomic_p)# 原子贡献
print(p)# 总能量


# 可视化
value = np.array(atomic_p.detach().cpu()).reshape(-1)
value = (value-np.min(value))/(np.max(value)-np.min(value)) * 200+50
value = np.int16(value)

setting = {
    'H':[0.3],
    'C':[0.3],
    'O':[0.3],
    'N':[0.3],
    'S':[0.3],
    'F':[0.3],
    'Cl':[0.3],
    'Br':[0.3],
    'I':[0.3]
}

def self_hex(n):
    return hex(n)[2:].zfill(2)

write('md_n.xyz', atom)

# 贡献图
view = py3Dmol.view(width=300, height=300)
view.addModel(open('md_n.xyz').read(), format='xyz')
for i,type in enumerate(atype[0]):
    view.setStyle({'index':i}, {'sphere': {'scale':setting[type][0],'color':'#'+self_hex(value[i])+self_hex(value[i])+self_hex(value[i])}})
view.zoomTo(animate=True)
view.show()

# 分子图
view_2 = py3Dmol.view(width=300, height=300)
view_2.addModel(open('md_n.xyz').read(), format='xyz')
view.setStyle({'sphere': {'scale': 0.3}})
view.zoomTo(animate=True)
view.show()

tensor([[-0.0793],
        [-0.0719],
        [-0.0185],
        [-0.0639],
        [-0.0534],
        [-0.0790],
        [-0.0493],
        [-0.0810],
        [-0.0507],
        [-0.0406],
        [-0.0422],
        [-0.0542],
        [-0.0519],
        [-0.0291],
        [-0.0398],
        [-0.0706],
        [-0.0292],
        [-0.0444],
        [-0.0393],
        [-0.0430],
        [-0.0618],
        [-0.0627],
        [-0.0488],
        [-0.0690],
        [-0.0792],
        [-0.0409],
        [-0.0677],
        [-0.0341],
        [-0.0245],
        [-0.0728],
        [-0.0795],
        [-0.0522],
        [-0.0562],
        [-0.0550],
        [-0.0145],
        [-0.0437],
        [-0.0222],
        [-0.0242],
        [-0.0311],
        [-0.0270],
        [-0.0774],
        [-0.0306],
        [-0.0527],
        [-0.0837],
        [-0.0562],
        [-0.0494],
        [-0.0430],
        [-0.0667],
        [-0.0699],
        [-0.0558],
        [-0.0422],
        [-0.0663],
        [-0.

IndexError: index 512 is out of bounds for axis 0 with size 512